# *Challenge 1*: A **kernel** methods / **DL** pipeline for the FashionMNIST dataset

Advanced Topics in Machine Learning -- Fall 2023, UniTS

<a target="_blank" href="https://colab.research.google.com/github/ganselmif/adv-ml-units/blob/main/notebooks/AdvML_Challenge_1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>


During this first *challenge* of the *Advanced Machine Learning Course*, you will experiment with the development of a data analysis pipeline based upon various techniques seen during the lectures so far: some **unsupervised** (*e.g.* *PCA*, *kernel-PCA*) and some others **supervised** (*e.g.* *kernel SVM* and *Artificial Neural Networks* for classification).

In [ ]:
## Import Libraries
import numpy as np

from tqdm.auto import trange

import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.utils.data import DataLoader, SubsetRandomSampler, TensorDataset
import torch.optim as optim

import torchvision
from torchvision import datasets, transforms


import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from sklearn.decomposition import PCA, KernelPCA
# from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score, accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC

## Used to save data into files
import pickle as pkl
import os

## The dataset

The dataset of interest for the *challenge* will be [*FashionMNIST*](https://github.com/zalandoresearch/fashion-mnist), an *MNIST*-like dataset of grayscale images of fashion items. Originally developed by Zalando Research in 2017 as a harder (yet *drop-in* compatible) replacement for the original *MNIST* dataset, it has been used in several papers and competitions since then.

Remember to consider the *dataset* mentioned below as the **training set** offered by *FashionMNIST*. Use the *test set* only at the end, to evaluate the overall accuracy of the pipeline!

In [ ]:
## Import train and test dataset, scale them and convert them to data loaders

train_dataset = datasets.FashionMNIST(
    root="./data",
    train=True,
    transform= transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(0, 1)]),
    download=True
)

test_dataset = datasets.FashionMNIST(
    root="./data",
    train=False,
    transform= transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(0, 1)]),
    download=True
)

train_loader = DataLoader(dataset = train_dataset,
                          batch_size = len(train_dataset),
                          shuffle = False)

test_loader = DataLoader(dataset = test_dataset,
                          batch_size = len(test_dataset),
                          shuffle = False)

In [ ]:
## Randomly select 10000 images from the training and test dataset

subset_size = 10000

## set a seed for randperm
th.manual_seed(42)

idx = th.randperm(len(train_dataset))[:subset_size]

sampler = SubsetRandomSampler(idx)

train_subset_loader = DataLoader(train_dataset, sampler=sampler)

idx = th.randperm(len(test_dataset))[:subset_size]

sampler = SubsetRandomSampler(idx)

test_subset_loader = DataLoader(train_dataset, sampler=sampler)

del(idx)
del(sampler)

In [ ]:
## Convert the images and their labels to numpy arrays and reshape them to vectors

labels_subset = []
train_subset = []
for batch in train_subset_loader:
    data, labels = batch
    
    train_subset.append(data.numpy().reshape(1, -1))
    labels_subset.append(labels.numpy())

train_subset_scaled = np.array(train_subset).reshape(subset_size, -1)
labels_subset = np.array(labels_subset)

In [ ]:
test_subset = []
labels_test = []

for batch in test_subset_loader:
    data, labels = batch
    
    test_subset.append(data.numpy().reshape(1, -1))
    labels_test.append(labels.numpy())

test_subset_scaled = np.array(test_subset).reshape(subset_size, -1)
labels_test = np.array(labels_test)

In [ ]:
# Creating dictionary of labels for better understanding
description = {0: "T-shirt/top", 
               1: "Trouser", 
               2: "Pullover", 
               3: "Dress", 
               4: "Coat", 
               5: "Sandal", 
               6: "Shirt", 
               7: "Sneaker", 
               8: "Bag", 
               9: "Ankle boot"}


In [ ]:
## Defining functions to save and load data from pickle files

def save_data(data, filename):
    if not os.path.exists(filename):
        with open(filename, "wb") as f:
            pkl.dump(data, f)

def load_data(filename):
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            data = pkl.load(f)
    return data

## Section 1: Understanding data geometry

Load the dataset in the most suitable form for the tasks that follow. Then, perform the following steps, with the goal of developing a geometric understanding of the dataset:

1. Perform a (linear) *PCA* on the dataset, and plot the first two (or three!) principal components along with the true label. Comment on data separation.

2. Perform a *kernel-PCA* on the dataset with a Gaussian kernel, and plot the first two (or three!) principal components along with the true label. Try to tune the dispersion parameter of the kernel to obtain a good separation of the data. Comment.

3. Perform another *kernel-PCA* on the dataset with another kernel of your own choice, and plot the first two (or three!) principal components along with the true label. Try to tune the degree of the polynomial kernel to obtain a good separation of the data. Comment.

Whenever suitable, try to complement your analysis with some graphs!

**IMPORTANT NOTICE**: As some of you have reported, performing *kernel PCA* on the entire *FashionMNIST* dataset can be memory- and time- demanding (as it scales with the square of the number of datapoints!). In case you want to reduce such requirements, you can either:
  - Reduce the number of datapoints on which to perform *kPCA*, *e.g.* by slicing the randomly-shuffled dataset *(most effective!)*;
  - Reduce the size of the images in the dataset (*i.e.* by dropping even/odd rows/columns, or performing local pooling) *(somehow effective: you may want to try it if you do not have access to powerful compute, but still enjoy challenges!)*.

In [ ]:
## Perform linear PCA and print its first 2 and 3 components


# ## Save data to files
# if os.path.exists("data_pca_linear.pickle"):
#      ## If the file exixts, load it
#     data_pca_linear = load_data("data_pca_linear.pickle")
    
# else:
#     ## If the file does not exixts, create it
    
model = PCA()
data_pca_linear = model.fit_transform(train_subset_scaled)
# save_data(data_pca_linear, "data_pca_linear.pickle")
        
# cmap = plt.get_cmap('jet', 10)
p = plt.scatter(data_pca_linear[:, 0], data_pca_linear[:, 1], c = labels_subset, marker='.')
cb = plt.colorbar(p)

cb.ax.set_title('Class', fontsize=11)
cb.ax.set_yticklabels(description.values())
plt.show()

del(model)
del(p)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Color the data points based on the labels
for i in range(3):
    p = ax.scatter(data_pca_linear[:, 0], data_pca_linear[:, 1], data_pca_linear[:, 2], c = labels_subset, marker='.')
    
    
cb = plt.colorbar(p)
cb.ax.set_title('Class', fontsize=11)
cb.ax.set_yticklabels(description.values())
ax.view_init(elev=90, azim=135)

del(fig)
del(ax)
del(p)
del(cb)

#### Comment
The data does not seem to be well separated, so finding the right hyperplane for classification will be hard.

### 1.2: Perform kernel PCA

In [ ]:
## Perform kernel pca using the RBF kernel
# if os.path.exists("data_pca_rbf.pickle"):
#      ## If the file exixts, load it
#     data_pca_rbf = load_data("data_pca_rbf.pickle")
    
# else:
    ## If the file does not exixts, create it
    
kernel_pca = KernelPCA(kernel="rbf", n_components = 3)
data_pca_rbf = kernel_pca.fit_transform(train_subset_scaled)

# save_data(data_pca_rbf, "data_pca_rbf.pickle")

In [ ]:
p = plt.scatter(data_pca_rbf[:, 0], data_pca_rbf[:, 1], c = labels_subset, marker='.')

cb = plt.colorbar(p)
cb.ax.set_title('Class', fontsize=11)
cb.ax.set_yticklabels(description.values())

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Color the data points based on the labels
for i in range(3):
    p = ax.scatter(data_pca_rbf[:, 0], data_pca_rbf[:, 1], data_pca_rbf[:, 2], c = labels_subset, marker='.')

cb = plt.colorbar(p)
cb.ax.set_title('Class', fontsize=11)
cb.ax.set_yticklabels(description.values())
ax.view_init(elev=90, azim=135)

ax.view_init(elev=0, azim=135)

### Perform parameter tuning

In [ ]:
## Choose the range of the parameter gamma
gammas = np.arange(1/784 - 5 * (1/784) * (1/10), 1/784 + 5 * (1/784) * (1/10), (1/784) * (1/10))


if os.path.exists("eigenvalues_rbf.pickle"):
     ## If the file exixts, load it
    eigenvalues_rbf = load_data("eigenvalues_rbf.pickle")
    
else:
    ## If the file does not exixts, create it

    ## Extract eigenvalues
    n_components = 3
    eigenvalues_rbf = np.empty((len(gammas), n_components))

    for i in range(len(gammas)):
        kernel_pca = KernelPCA(kernel="rbf", n_components = n_components, gamma = gammas[i])
        eigenvalues_rbf[i] = kernel_pca.fit(train_subset_scaled).eigenvalues_
        
    save_data(eigenvalues_rbf, "eigenvalues_rbf.pickle")



# Plot the results
fig, axs = plt.subplots(2, 5, figsize=(30, 10))

# Create 10 random plots
for i, ax in enumerate(axs.flat):
    # Generate random data
    x = np.arange(1, len(eigenvalues_rbf[i, :]) + 1, 1)
    # Plot the data on the corresponding axis
    ax.bar(x, eigenvalues_rbf[i, :])
    ax.set_ylim(0, 1000)
    # ax.set_xlabel('Component')
    # ax.set_ylabel('Eigenvalue')
    ax.set_title('Gamma = ' + str(gammas[i]))

### 1.3 Perform kPCA using another kernel

In [ ]:
## Try kernel poly

if os.path.exists("data_pca_poly.pickle"):
     ## If the file exixts, load it
    data_pca_poly = load_data("data_pca_poly.pickle")
    
else:
    ## If the file does not exixts, create it
    
    kernel_pca = KernelPCA(kernel="poly", n_components = 3)

    data_pca_poly = kernel_pca.fit_transform(train_subset_scaled)
    
    save_data(data_pca_poly, "data_pca_poly.pickle")
    


p = plt.scatter(data_pca_poly[:, 0], data_pca_poly[:, 1], c = labels_subset, marker='.')

cb = plt.colorbar(p)
cb.ax.set_title('Class', fontsize=11)
cb.ax.set_yticklabels(description.values())

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Color the data points based on the labels
for i in range(3):
    p = ax.scatter(data_pca_poly[:, 0], data_pca_poly[:, 1], data_pca_poly[:, 2], c = labels_subset, marker='.')

cb = plt.colorbar(p)
cb.ax.set_title('Class', fontsize=11)
cb.ax.set_yticklabels(description.values())
ax.view_init(elev=45, azim=45)

In [ ]:
## Try kernel sigmoid

# if os.path.exists("data_pca_sigmoid.pickle") and os.path.exists("eigenvalues_sigmoid.pickle"):
    
#      ## If the file exixts, load it
#     data_pca_sigmoid = load_data("data_pca_sigmoid.pickle")
#     eigenvalues_sigmoid = load_data("eigenvalues_sigmoid.pickle")
    
# else:
    ## If the file does not exixts, create it
    
kernel_pca = KernelPCA(kernel="sigmoid", n_components = 10)

data_pca_sigmoid = kernel_pca.fit_transform(train_subset_scaled)

# save_data(data_pca_sigmoid, "data_pca_sigmoid.pickle")

eigenvalues_sigmoid = kernel_pca.eigenvalues_

# save_data(eigenvalues_sigmoid, "eigenvalues_sigmoid.pickle")

p = plt.scatter(data_pca_sigmoid[:, 0], data_pca_sigmoid[:, 1], c = labels_subset, marker='.')

cb = plt.colorbar(p)
cb.ax.set_title('Class', fontsize=11)
cb.ax.set_yticklabels(description.values())
ax.view_init(elev=90, azim=135)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Color the data points based on the labels
for i in range(3):
    p = ax.scatter(data_pca_sigmoid[:, 0], data_pca_sigmoid[:, 1], data_pca_sigmoid[:, 2], c = labels_subset, marker='.')

cb = plt.colorbar(p)
cb.ax.set_title('Class', fontsize=11)
cb.ax.set_yticklabels(description.values())
ax.view_init(elev=0, azim=180)

## Section 2: Bridging *unsupervised* and *supervised*

Choose one of the results obtained in the previous section (you should choose the one better explaining data geometry), and ignore the true labels. Then, perform the following steps:

1. Considering only the first $10$ components of the *(kernel-)PCA* and try to assign $10$ labels to the resulting datapoints. Choose the approach you deem most suitable. Comment on the results, by considering:
    
    a. How well does the label-assignment just performed reflect the true labels?

    b. Does the number of components used ($10$) reflect the actual *knee-* or *gap-* *point* of the spectrum associated to the principal components?

Whenever suitable, try to complement your analysis with some graphs!

In [ ]:
labels_Kmeans = KMeans(n_clusters = 10).fit(data_pca_sigmoid).labels_

labels_Spectral = SpectralClustering(n_clusters = 10, affinity='nearest_neighbors').fit(data_pca_sigmoid).labels_

labels_Gaussian = GaussianMixture(n_components = 10).fit(data_pca_sigmoid).predict(data_pca_sigmoid)

labels = np.array([labels_subset.reshape(subset_size), labels_Kmeans, labels_Spectral, labels_Gaussian])

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)

title_names = ["Original", "KMeans", "Spectral Clustering", "Gaussian Mixture"]

for ax, i in zip(axs.flat, range(4)):
    ax.scatter(data_pca_sigmoid[:, 0], data_pca_sigmoid[:, 1], c = labels[i, :], marker='.')
    ax.set_title(title_names[i])

In [ ]:
## Adjusted Rand Index

ARI = np.empty(3)

for i in range(3):
    ARI[i] = adjusted_rand_score(labels[0, :], labels[i + 1, :])
    print("Adjusted Rand Index for " + title_names[i + 1] + ": " + str(ARI[i]))

#### a
As we can see, label assignment performed poorly. This, probably, because the clusters are very close to each other and not clearly separated.

#### b
As we can see from the plot below, there is a clear elbow on the third component. This suggests that 10 does not reflect the actual knee point of the spectrum of the principal components.

In [ ]:
plt.plot(np.arange(1, len(eigenvalues_sigmoid) + 1, 1), eigenvalues_sigmoid)

plt.xlabel('Component')
plt.ylabel('Eigenvalue')

## Section 3: (Supervised) classification

Consider the dataset composed of the original images, with the label assigned in the previous section (regardless of its actual match with the true label!). Then, define and learn a classifier that can predict the label of a new image. Specifically:

1. Learn a *kernel-SVM* on the data/label pairs. The choice of the kernel and its hyperparameters is up to your experimentation and time availability. Comment on your choices and results.

2. Learn a *fully-connected NN* on the data/label pairs. The choice of the architecture and its hyperparameters is up to your experimentation and time availability: show at least two different hyperparameter configurations, and comment on the results.

3. Learn a *CNN* on the data/label pairs. The choice of the architecture and its hyperparameters is up to your experimentation and time availability. Comment on the results with special respect to the FCN architecture.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_subset_scaled, labels_Gaussian, test_size=0.3, random_state=42)

#### 1: kernel SVM with different kernels

In [ ]:
classifier = SVC(kernel = "linear").fit(x_train, y_train)

label_predict_SVC_linear = classifier.predict(x_test)

acc_linear = accuracy_score(y_test, label_predict_SVC_linear)

In [ ]:
classifier = SVC(kernel = "sigmoid").fit(x_train, y_train)

label_predict_SVC_sigmoid = classifier.predict(x_test)

acc_sigmoid = accuracy_score(y_test, label_predict_SVC_sigmoid)

In [ ]:
classifier = SVC(kernel = "rbf").fit(x_train, y_train)

label_predict_SVC_rbf = classifier.predict(x_test)

acc_rbf = accuracy_score(y_test, label_predict_SVC_rbf)

In [ ]:
print(f"Accuracy: linear kernel: {acc_linear:.4f} | sigmoid kernel: {acc_sigmoid:.4f} | rbf kernel: {acc_rbf:.4f}")

#### 2: Fully Connected NN

In [ ]:
## Pass data to tensors

data_train = TensorDataset(Tensor(x_train.reshape(-1, 1, 28, 28)), th.tensor(y_train, dtype = th.long))
data_train_loader = DataLoader(dataset = data_train, batch_size = 64, shuffle = True)



data_test = TensorDataset(Tensor(x_test.reshape(-1, 1, 28, 28)), th.tensor(y_test, dtype = th.long))
data_test_loader = DataLoader(dataset = data_test, batch_size = 64, shuffle = True)

In [ ]:
# Define some functions needed to calculate the accuracy

def get_batch_accuracy(logit, target):
    corrects = (th.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects / target.size(0)
    return accuracy.item()


def get_test_stats(model, criterion, test_loader, device):
    test_acc, test_loss = 0.0, 0.0
    for i, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_loss += criterion(outputs, labels).item()
        test_acc += get_batch_accuracy(outputs, labels)
        return test_loss, test_acc

In [ ]:
# Define function used to train the model

def train_model(epochs, train_loader, test_loader, criterion, optimizer, device, model):
    _batch_losses = []
    
    
    for _ in trange(epochs):
        _model = model.train()

        # Actual (batch-wise) training step
        for _, (_images, _labels) in enumerate(train_loader):
            _images = _images.to(device)
            _labels = _labels.to(device)

            _logits = _model(_images)
            _loss = criterion(_logits, _labels)
            _batch_losses.append(_loss.item())  # Store the loss for plotting, per batch

            optimizer.zero_grad()
            _loss.backward()
            optimizer.step()

        _model.eval()
        
    _trained_loss, _trained_acc = get_test_stats(_model, criterion, test_loader, device)

    # print(f"Accuracy: {_trained_acc} | Loss: {_trained_loss}")
    
    return _trained_acc, _trained_loss, _model

In [ ]:
def get_predicted_labels(model, test_data, device):
    test_data = th.tensor(x_test.reshape(-1, 1, 28, 28))

    model = model.eval()
    model = model.to(device)

    labels = []
    with th.no_grad():
        for i in range(test_data.shape[0]):
            data = test_data[i].reshape(1, 1, 28, 28)
            pred = model(data.to(device))
            labels.append(th.argmax(pred).item())
            
    return np.array(labels)

In [ ]:
# Define the Fully Connected Neural Network

class FullyConnectedNN(nn.Module):
    def __init__(self, image_dim, n_classes):
        
        super(FullyConnectedNN, self).__init__()
        
        self.fc = nn.Linear(in_features = image_dim,
                            out_features = n_classes)
    
    def forward(self, x):
        x = x.flatten(start_dim = 1)
        x = self.fc(x)
        x = F.relu(x)
        x = F.log_softmax(x, dim = 1)
        return x

In [ ]:
## initialise model
model_FCNN = FullyConnectedNN(image_dim = 28 * 28, n_classes = 10)

## Choose optimizer
optimizer = th.optim.SGD(model_FCNN.parameters(), lr=0.0001, momentum=0)

# Choose loss
criterion = nn.CrossEntropyLoss()
# criterion = nn.NLLLoss()

# Device selection
device = th.device("cuda" if th.cuda.is_available() else "cpu")
model_FCNN = model_FCNN.to(device)
print(f"Using device: {device}")



# Untrained Accurac
model_FCNN = model_FCNN.eval()

untrained_loss, untrained_acc = get_test_stats(model = model_FCNN,
                                               criterion = criterion,
                                               test_loader = data_test_loader,
                                               device = device)

model_FCNN = model_FCNN.train()

print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")


# Train the model
trained_acc, trained_loss, model_FCNN = train_model(epochs = 3,
                                                             train_loader = data_train_loader,
                                                             test_loader = data_test_loader,
                                                             criterion = criterion,
                                                             optimizer = optimizer,
                                                             device = device,
                                                             model = model_FCNN)
print(f"Accuracy: {trained_acc} | Loss: {trained_loss}")

In [ ]:
FCNN_labels = get_predicted_labels(model = model_FCNN, test_data = x_test, device = device)

accuracy_score(FCNN_labels, y_test)

#### 3: Convolutional Neural Network

In [ ]:
class CNN(nn.Module):
    def __init__(self, n_classes, kernel_size, input_size, layers, pooling):
        super(CNN, self).__init__()
        
        self._n_classes = n_classes
        
        self._padding = 0 # Default value
        
        self._stride = 1 # Default value
        
        self._stride_inv = 1 / self._stride
        
        self._kernel = kernel_size
        
        self._dimensions = input_size
        
        self._print_dimensions = False
        
        self._layers = layers
        
        self._pooling = pooling
        
        if self._layers >= 1:   
            self.conv1, self.bn1, self._dimensions = self.conv_layer(self._kernel)
         
        if self._layers >= 2:
            self.conv2, self.bn2, self._dimensions = self.conv_layer(self._kernel)
            
        if self._pooling == True:
            self.pool = nn.MaxPool2d(kernel_size = self._kernel, stride = self._stride)
            if self._print_dimensions == True:
                print("Defined pool")
        
            self._dimensions = [self._dimensions[0],
                                self._dimensions[1] ,
                                (self._dimensions[2] - self._kernel) * self._stride_inv + 1,
                                (self._dimensions[3] - self._kernel) * self._stride_inv + 1
                                ]
        
        if self._layers >= 3: 
            self.conv3, self.bn3, self._dimensions = self.conv_layer(self._kernel)
            
        
        if self._layers >= 4: 
            self.conv4, self.bn4, self._dimensions = self.conv_layer(self._kernel)
        
        
        self.fc1 = nn.Linear(in_features = int(self._dimensions[2] * self._dimensions[3]), out_features = self._n_classes)
        if self._print_dimensions == True:
            print("Defined fc1")
            
    def conv_layer(self, kernel_size):
        conv = nn.Conv2d(in_channels = self._dimensions[1], out_channels = self._dimensions[1], kernel_size = kernel_size)
        
        dimensions = [self._dimensions[0],
                            self._dimensions[1] ,
                            (self._dimensions[2] - self._kernel + 2 * self._padding) * self._stride_inv + 1,
                            (self._dimensions[3] - self._kernel + 2 * self._padding) * self._stride_inv + 1
                            ]
        
        bn = nn.BatchNorm2d(self._dimensions[1])
        
        self.relu = nn.ReLU()
        
        return conv, bn, dimensions
    
    
    

    def apply_conv_layer(self, input, conv, bn):
        output = conv(input)
        output = bn(output) 
        output = F.relu(output)
        
        return output 
    
        
    def forward(self, input):
        
        if self._print_dimensions == True:
            print(f"Input size: {input.size()}")
            
        output = input
        
        if self._layers >= 1: 
            output = self.apply_conv_layer(output, self.conv1, self.bn1) 
        
        if self._layers >= 2:
            output = self.apply_conv_layer(output, self.conv2, self.bn2)
        
        
        if self._pooling == True:
            output = self.pool(output)    
            if self._print_dimensions == True:
                print(f"Applied pool, size: {output.size()}")
        
        
        if self._layers >= 3:
            output = self.apply_conv_layer(output, self.conv3, self.bn3)
        
        
        if self._layers >= 4:
            output = self.apply_conv_layer(output, self.conv4, self.bn4)
        

        output = output.view(output.shape[0], -1)
        if self._print_dimensions == True:
            print(f"Applied view, size: {output.size()}")
        
        output = self.fc1(output)
        
        if self._print_dimensions == True:
            print(f"Applied fc1, size: {output.size()}")
        
        
        return F.log_softmax(output, dim = 1)

In [ ]:
# Device selection
model = CNN(n_classes = 10, kernel_size = 3, input_size=[7000, 1, 28, 28], layers = 3, pooling = True)

criterion = nn.CrossEntropyLoss()

# criterion = nn.NLLLoss()

optimizer = optim.Adam(model.parameters(), lr = 0.0001, weight_decay = 0.0001)

# optimizer = th.optim.SGD(model.parameters(), lr=0.01, momentum=0)

device = th.device("cuda" if th.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using device: {device}")

model = model.eval()

untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)

model = model.train()

print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")

trained_acc, trained_loss, model_CNN = train_model(epochs = 3,
                                                             train_loader = data_train_loader,
                                                             test_loader = data_test_loader,
                                                             criterion = criterion,
                                                             optimizer = optimizer,
                                                             device = device,
                                                             model = model)
print(f"Accuracy: {trained_acc} | Loss: {trained_loss}")

In [ ]:
CNN_labels = get_predicted_labels(model = model_CNN, test_data = x_train, device = device)
accuracy_score(CNN_labels, y_test)

### Section 5: A *fully-supervised* approach

Repeat the steps of *Section 3* using the true labels of the dataset. Comment on the results, and draw a comparison between such results and those obtained from the previous *hybrid* pipeline.

In [ ]:
#Importing the dataset again

subset_size = 64

train_dataset = datasets.FashionMNIST(
    root="./data",
    train=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=0, std=1)]),
    download=True
)

test_dataset = datasets.FashionMNIST(
    root="./data",
    train=False,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=0, std=1)]),
    download=True
)

train_loader = DataLoader(dataset = train_dataset, shuffle=True, batch_size = subset_size)
test_loader = DataLoader(dataset = test_dataset, shuffle=True, batch_size = subset_size)

Implementing linear and rbf SVM and confronting it with the sklearn implementation

In [ ]:
# class LinearSVM(nn.Module):
#     def __init__(self, n_classes, image_size):
#         super(LinearSVM, self).__init__()

#         self._linear = nn.Linear(image_size, n_classes)
            
        
#     def forward(self, x):
#         x = x.flatten(start_dim = 1)
#         x = self._linear(x)
#         x = F.log_softmax(x, dim = 1)
#         return x


# model = LinearSVM(n_classes = 10, image_size = 28 * 28)


In [ ]:
# device = th.device("cuda" if th.cuda.is_available() else "cpu")
# model = model.to(device)
# print(f"Using device: {device}")

# # criterion = nn.CrossEntropyLoss()

# criterion = nn.NLLLoss()

# optimizer = th.optim.SGD(model.parameters(), lr=0.01, momentum=0)


# model = model.eval()

# untrained_loss, untrained_acc = get_test_stats(model, criterion, test_loader, device)

# model = model.train()

# print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")

# trained_acc, trained_loss, model, batch_losses = train_model(epochs = 3,
#                                                              train_loader = train_loader,
#                                                              test_loader = test_loader,
#                                                              criterion = criterion,
#                                                              optimizer = optimizer,
#                                                              device = device,
#                                                              model = model)
# print(f"Accuracy: {trained_acc} | Loss: {trained_loss}")

In [ ]:
classifier = SVC(kernel = "linear").fit(train_subset_scaled, labels_subset)

label_predict = classifier.predict(test_subset_scaled)
accuracy_score(labels_test, label_predict)

In [ ]:
classifier = SVC(kernel = "rbf").fit(train_subset_scaled, labels_subset)

label_predict = classifier.predict(test_subset_scaled)
accuracy_score(labels_test, label_predict)

In [ ]:
train_subset_scaled.shape

#### 3.2: Fully Connected NN

In [ ]:
model = FullyConnectedNN(image_dim = 28 * 28, n_classes = 10)

# Device selection
device = th.device("cuda" if th.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using device: {device}")

criterion = nn.NLLLoss()

optimizer = th.optim.SGD(model.parameters(), lr=0.01, momentum=0)

model = model.eval()

untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)

model = model.train()

print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")


trained_acc, trained_loss, model = train_model(epochs = 3,
                                                             train_loader = train_loader,
                                                             test_loader = test_loader,
                                                             criterion = criterion,
                                                             optimizer = optimizer,
                                                             device = device,
                                                             model = model)
print(f"Accuracy: {trained_acc} | Loss: {trained_loss}")

In [ ]:
# batch_losses = []

# for epoch in trange(2):
#     model = model.train()

#     # Actual (batch-wise) training step
#     for i, (images, labels) in enumerate(train_loader):
#         images = images.to(device)
#         labels = labels.to(device)

#         logits = model(images)
#         loss = criterion(logits, labels)

#         batch_losses.append(loss.item())  # Store the loss for plotting, per batch

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     model.eval()

In [ ]:
# # Get test stats after training
# trained_loss, trained_acc = get_test_stats(model, criterion, test_loader, device)

# print(f"Accuracy: {trained_acc} | Loss: {trained_loss}")

In [ ]:
# # Plot losses
# xrange = range(len(batch_losses))
# _ = plt.plot(xrange, batch_losses)
# _ = plt.scatter((xrange[0], xrange[-1]), (untrained_loss, trained_loss), color="red")
# _ = plt.legend(["Train Loss (batch)", "Test Loss (dataset)"], loc="upper right")
# _ = plt.xlabel("Number of batches seen")
# _ = plt.ylabel("NLL Loss")

#### 3.2: Convolutional NN

In [ ]:
# Device selection
model = CNN(n_classes = 10, kernel_size = 2, input_size = [64, 1, 28, 28], layers = 4, pooling = True)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr = 0.0001, weight_decay = 0.0001)

# optimizer = th.optim.SGD(model.parameters(), lr=0.01, momentum=0)

device = th.device("cuda" if th.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using device: {device}")

model = model.eval()

untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)

model = model.train()

print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")

In [ ]:
batch_losses = []
EPOCHS = 2

for epoch in trange(EPOCHS):
    model = model.train()

    # Actual (batch-wise) training step
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        logits = model(images)
        loss = criterion(logits, labels)

        batch_losses.append(loss.item())  # Store the loss for plotting, per batch

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()

In [ ]:
# Get test stats after training
trained_loss, trained_acc = get_test_stats(model, criterion, test_loader, device)

print(f"Accuracy: {trained_acc} | Loss: {trained_loss}")